# magics

> Fill in a module description here

In [1]:
#| default_exp magics

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export

import os
import openai
from IPython.display import display, Markdown

openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
#| export

class OpenAIAPI():
    def __init__(self):
        self.reset_context()

    def reset_context(self):
        self.context = [
            {
                'role':'system',
                'content': 'You are an expert programmer helping out a friend. Your friend is using Python in Jupyter Notebook. Give a succinct answer that a programmer with one year of professional experience would easily understand.'
            }
        ]

    def get_completion(self, prompt, new_conversation=True):
        if new_conversation:
            self.reset_context()
        
        self.context.append(
            {
                'role':'user',
                'content': prompt
            }
        )

        response = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo",
            messages = self.context
        )
        
        completion = self.extract_completion(response)
        self.extend_context(response)
        self.prune_context()
        self.display_completion(completion)
        
        self.last_response = response  # useful for debugging
    
    def extract_completion(self, response):
        return response['choices'][0].message.content.strip()
    
    def extend_context(self, response):
        self.context.append(response['choices'][0].message.to_dict())
    
    def prune_context(self):
        # Prune context to under 2200 words. That should be ~3000 tokens, leaving room for the prompt and completion.
        pruned_context = []
        word_count = 0
        while self.context:
            last_message = self.context.pop()
            word_count += len(last_message['content'].split())
            if word_count < 2200:
                pruned_context.append(last_message)
            else:
                break
        pruned_context.reverse()

    def display_completion(self, completion):
        display(Markdown(completion))

open_ai_api = OpenAIAPI()

In [5]:
#| export

def ai_ask(line, cell):
    open_ai_api.get_completion(cell)

def ai_continue(line, cell):
    open_ai_api.get_completion(cell, False)
    
def load_ipython_extension(ipython):
    ipython.register_magic_function(ai_ask, magic_kind='cell', magic_name='ai_ask')
    ipython.register_magic_function(ai_continue, magic_kind='cell', magic_name='ai_continue')

In [6]:
#| hide
import nbdev; nbdev.nbdev_export()